In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import datetime as dt
import mplfinance as mpf
import time
import yfinance as yf

import os
from os import listdir
from os.path import isfile, join

In [2]:
#default Values
path ="/workspaces/Quantitative-Finance/Data/"

S_year = 2019
S_month = 1
S_day = 1
S_date_str = f"{S_year}-{S_month}-{S_day}"
S_date_datetime = dt.datetime(S_year,S_month,S_day)

E_year = 2024
E_month = 2
E_day = 2
E_date_str = f"{E_year}-{E_month}-{E_day}"
E_date_datetime = dt.datetime(E_year,E_month,E_day)


In [3]:
# # get stock file names in a list
# files = [x for x in listdir(path) if isfile(join(path,x))]
# tickers = [os.path.splitext(x)[0] for x in files]
# # create dataframe from our list
# stock_df = pd.DataFrame(tickers,columns=["tickers"])

# return dataframe from csv
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(path+ticker+".csv")
    except FileNotFoundError:
        print("File doesn't exist")
    else:
        return df
    

# Save dataframe to csv
def save_dataframe_to_csv(df,ticker):
    df.to_csv(path+ticker+".csv")

# return on investment over time
def get_roi(df):
    df['Date'] = pd.to_datetime(df['Date'])
    start_val = df[df['Date'] == S_date_str]['Adj Close'][0]
    end_val = df[df['Date'] == E_date_str]['Adj Close'][0]
    print("Initial Price:",start_val)
    print("Final Price:",end_val)
    roi = (end_val - start_val) / start_val
    return roi

# get coefficient of variance
def get_cov(stock_df):
    for stock in stock_df:
        mean,sd = stock_df['Adj Close'].mean(),stock_df['Adj Close'].std()
        cov = sd / mean
        return cov
    

# Merge Multiple stock in on df
def merge_df_by_column_name(col_name,*tickers):
    
    mult_df = pd.DataFrame()
    start = S_date_datetime
    end = E_date_datetime
    
    for x in tickers:
        mult_df[x] = get_df_from_csv(x)[col_name]
    return mult_df



In [5]:
# Get Sector Stocks
sec_df = pd.read_csv('/workspaces/Quantitative-Finance/Basic/stock_sectors.csv')
sec_df

indus_df = sec_df.loc[sec_df['Sector'] == "Industrials"]
health_df = sec_df.loc[sec_df['Sector'] == "Health Care"]
it_df = sec_df.loc[sec_df['Sector'] == "Information Technology"]
comm_df = sec_df.loc[sec_df['Sector'] == "Communication Services"]
staple_df = sec_df.loc[sec_df['Sector'] == "Consumer Staples"]
discretion_df = sec_df.loc[sec_df['Sector'] == "Consumer Discretionary"]
utility_df = sec_df.loc[sec_df['Sector'] == "Utilities"]
financial_df = sec_df.loc[sec_df['Sector'] == "Financials"]
material_df = sec_df.loc[sec_df['Sector'] == "Materials"]
restate_df = sec_df.loc[sec_df['Sector'] == "Real Estate"]
energy_df = sec_df.loc[sec_df['Sector'] == "Energy"]

In [32]:
# returns a df with roi for all stocks
def get_rois_for_stocks(stock_df):
    tickers = []
    rois = []
    for index,row in stock_df.iterrows():
        df = get_df_from_csv(row['Symbol'])
        if df is None:
            pass
        else:
            tickers.append(row['Symbol'])
            df = df.set_index(['Date'])
            roi = get_roi(df)
            rois.append(roi)
    return pd.DataFrame({"Tickers":tickers, "ROI":rois})

In [33]:
industrial = get_rois_for_stocks(indus_df)
industrial

KeyError: 'Date'

In [26]:
for index,row in indus_df.iterrows():
        df = get_df_from_csv(row['Symbol'])
        if df is None:
                pass
        else:
               print(index)
               df = df.set_index(['Date'])

0
1
16
20
29
37
71
80
File doesn't exist
91
109
126
131
132
138
139
153
161
168
172
183
188
190
200
201
209
210
211
230
File doesn't exist
237
239
241
242
245
257
File doesn't exist
262
265
File doesn't exist
281
287
299
331
334
336
347
File doesn't exist
353
355
360
386
File doesn't exist
395
397
398
399
400
414
416
417
432
437
File doesn't exist
448
459
460
461
462
469
479
480
483
499


In [27]:
df = df.set_index(['Date'])
df

,Unnamed: 0,Adj Close,daily_return
Date,,,
2017-01-03,0,46.964317,NaN
2017-01-04,1,47.664280,0.014904
2017-01-05,2,47.229168,-0.009129
2017-01-06,3,46.898098,-0.007010
2017-01-09,4,46.699455,-0.004236
...,...,...,...
2021-08-13,1161,131.529999,0.003356
2021-08-16,1162,132.929993,0.010644
2021-08-17,1163,131.800003,-0.008501
